<a href="https://colab.research.google.com/github/kacychou/MLplayground/blob/main/Sentiment_Analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:13:14tcmalloc: large alloc 1147494400 bytes == 0x557ca631e000 @  0x7fe244060615 0x557c6bcfb3bc 0x557c6bddc18a 0x557c6bcfe1cd 0x557c6bdf0b3d 0x557c6bd72458 0x557c6bd6d02f 0x557c6bcffaba 0x557c6bd722c0 0x557c6bd6d02f 0x557c6bcffaba 0x557c6bd6ecd4 0x557c6bdf1986 0x557c6bd6e350 0x557c6bdf1986 0x557c6bd6e350 0x557c6bdf1986 0x557c6bd6e350 0x557c6bcfff19 0x557c6bd43a79 0x557c6bcfeb32 0x557c6bd721dd 0x557c6bd6d02f 0x557c6bcffaba 0x557c6bd6ecd4 0x557c6bd6d02f 0x557c6bcffaba 0x557c6bd6deae 0x557c6bcff9da 0x557c6bd6e108 0x557c6bd6d02f
     |█████████████████               | 1055.7 MB 1.1 MB/s eta 0:13:34tcmalloc: large alloc 1434370048 bytes == 0x557cea974000 @  0x7fe244060615 0x557c6bcfb3bc 0x557c6bddc18a 0x557c6bcfe1cd 0x557c6bdf0b3d 0x557c6bd72458 0x557c6bd6d02f 0x557c6bcffaba 0x557c6bd722c0 0x557c6bd6d02f 0x557c6bcffaba 0x557c6bd6ecd4 0x557c6bdf1986 0x557c6bd6e3

In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy


     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.8 MB 32.4 MB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [43]:
#Step 0
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch 
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [9]:
#Step 1 Instantiate model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
#from_pretrained method loads in the model

In [11]:
#Step 2 Encode and calculate sentiment
tokens = tokenizer.encode('I hate this, absolutely the worst', return_tensors ='pt')
#return_tensors = pytorch

In [13]:
#pass it the model
result = model(tokens)

In [14]:
result
#result returns a one-hot encoded list of scores
#The position with the highest score represents the sentiment rating
#In this case position 0 = rating of 1

SequenceClassifierOutput([('logits',
                           tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
                                  grad_fn=<AddmmBackward0>))])

In [15]:
torch.argmax(result.logits)

tensor(0)

In [17]:
int(torch.argmax(result.logits))+1

1

In [22]:
#Step 2b Encode and calculate sentiment - happy review
tokens = tokenizer.encode('I love this, absolutely the best', return_tensors ='pt')
result = model(tokens)
int(torch.argmax(result.logits))+1

5

In [ ]:
#Step 3 - Collect reviews

In [36]:
#using regex to extract the classes of reviews on yelp
r=requests.get('https://www.yelp.com/biz/%E6%BE%B3%E6%B4%B2%E7%89%9B%E5%A5%B6%E5%85%AC%E5%8F%B8-%E9%A6%99%E6%B8%AF-2')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*') #parse regex to soup
results=soup.find_all('p',{'class':regex}) #find all that is wrapped by a p tag and has a class of "comments" - find class that matches regex

In [41]:
reviews = [ result.text for result in results] #extract results using a list comprehension


In [42]:
reviews 

['Amazing local food and very quick service!Probably the best breakfast deal for good food in town',
 "On food alone, this place is great. Having been here for many years, they really know what they are doing when it comes to food. I would personally recommend the egg sandwiches, since that's what they are really known for. However, a lot of people also like coming here for the boiled milk, among many other items. Overall, I don't think you can go wrong here for any menu item.The big problem here is the service. I wouldn't say it's that bad, in the sense that you know what you are getting when you come here: curt responses and an expectation that you will not wiffle waffle when it comes to placing your order. They are in a hurry here and rely on turnover to make money. So, prepare for that.The other issue is that this place is constantly closed. Due to COVID paranoia this place frequently just shuts down for days on end for no seeming reason. The alleged reason is to 'protect staff', b

In [45]:
#Step 4 - Load reviews into dataframe and score
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [47]:
df.head

<bound method NDFrame.head of                                              reviews
0  Amazing local food and very quick service!Prob...
1  On food alone, this place is great. Having bee...
2  This is a long overdue review when I visited b...
3  Please don't be alarmed if you're rushed to or...
4  This is a very busy restaurant that you'd hone...
5  This is undeniably the absolute BEST eggs I ha...
6  I came here because of the hype if I'm being h...
7  Came here on a Sunday morning and waited aroun...
8  I came for the French Toast and got exactly wh...
9  ADC is now a mainstream tourist attraction. Th...>

In [48]:
reviews

['Amazing local food and very quick service!Probably the best breakfast deal for good food in town',
 "On food alone, this place is great. Having been here for many years, they really know what they are doing when it comes to food. I would personally recommend the egg sandwiches, since that's what they are really known for. However, a lot of people also like coming here for the boiled milk, among many other items. Overall, I don't think you can go wrong here for any menu item.The big problem here is the service. I wouldn't say it's that bad, in the sense that you know what you are getting when you come here: curt responses and an expectation that you will not wiffle waffle when it comes to placing your order. They are in a hurry here and rely on turnover to make money. So, prepare for that.The other issue is that this place is constantly closed. Due to COVID paranoia this place frequently just shuts down for days on end for no seeming reason. The alleged reason is to 'protect staff', b

In [49]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [61]:
sentiment_score(df['reviews'].iloc[0])

5

In [66]:
df['sentiment']=df['reviews'].apply(lambda x: sentiment_score(x[:512])) #grab first 512 tokens for each review

In [67]:
df

,reviews,sentiment
0,Amazing local food and very quick service!Prob...,5
1,"On food alone, this place is great. Having bee...",3
2,This is a long overdue review when I visited b...,5
3,Please don't be alarmed if you're rushed to or...,5
4,This is a very busy restaurant that you'd hone...,5
5,This is undeniably the absolute BEST eggs I ha...,5
6,I came here because of the hype if I'm being h...,4
7,Came here on a Sunday morning and waited aroun...,5
8,I came for the French Toast and got exactly wh...,5
9,ADC is now a mainstream tourist attraction. Th...,4
